In [1]:
import sklearn_crfsuite

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from helper.dataset_reader import read_tsv
from helper.features import *

# import warnings filter
from warnings import simplefilter

# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [2]:
data = read_tsv('../dataset/all-merged-data-140422.tsv')

In [3]:
X = []
y = []

for tokens, tags in data[0]:
    X.append(tokens)
    y.append(tags)

In [4]:
X_train_, X_test_, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [5]:
X_train = []
for token in X_train_:
    X_train.append(feature_extraction(tokens, 5))

In [6]:
X_train[0]

[{'n_gram_0': 'Mau',
  'is_alpha': True,
  'is_numeric': False,
  'is_capital': False,
  'contains_alpha': True,
  'contains_numeric': False,
  'contains_aphostrope': False},
 {'n_gram_0': 'diblock',
  'is_alpha': True,
  'is_numeric': False,
  'is_capital': False,
  'contains_alpha': True,
  'contains_numeric': False,
  'contains_aphostrope': False,
  'n_gram_1': 'ibloc'},
 {'n_gram_0': 'tapi',
  'is_alpha': True,
  'is_numeric': False,
  'is_capital': False,
  'contains_alpha': True,
  'contains_numeric': False,
  'contains_aphostrope': False},
 {'n_gram_0': 'masih',
  'is_alpha': True,
  'is_numeric': False,
  'is_capital': False,
  'contains_alpha': True,
  'contains_numeric': False,
  'contains_aphostrope': False},
 {'n_gram_0': 'temen',
  'is_alpha': True,
  'is_numeric': False,
  'is_capital': False,
  'contains_alpha': True,
  'contains_numeric': False,
  'contains_aphostrope': False},
 {'n_gram_0': 'tapi',
  'is_alpha': True,
  'is_numeric': False,
  'is_capital': False,
  'co

In [7]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',  # for gradient descent for optimization and getting model parameters
    c1=0.1, # Coefficient for Lasso (L1) regularization
    c2=0.1, # Coefficient for Ridge (L2) regularization
    max_iterations=100, # The maximum number of iterations for optimization algorithms, iteration for the gradient descent optimization
    all_possible_transitions=True   # Specify whether CRFsuite generates transition features that do not even occur in the training data
)

In [8]:
X_train = []
for tokens in X_train_:
    X_train.append(feature_extraction(tokens, 5))

#try:
crf.fit(X_train, y_train)
#except AttributeError:
#    pass

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [9]:
labels = ['ID', 'JV', 'EN', 'O', 'MIX-ID-EN', 'MIX-ID-JV', 'MIX-JV-EN']

In [10]:
X_test = []
for tokens in X_test_:
    X_test.append(feature_extraction(tokens, 5))

y_pred = crf.predict(X_test)
print('Testing Data Performance')
flat_y = [item for y_ in y_test for item in y_]
flat_y_pred = [item for y_pred_ in y_pred for item in y_pred_]
print(classification_report(flat_y, flat_y_pred, labels=labels))

Testing Data Performance
              precision    recall  f1-score   support

          ID       0.89      0.97      0.93     22317
          JV       0.86      0.83      0.85      4915
          EN       0.92      0.82      0.87      5005
           O       0.97      0.87      0.92     10161
   MIX-ID-EN       0.98      0.83      0.90       976
   MIX-ID-JV       0.98      0.83      0.90       289
   MIX-JV-EN       0.96      0.82      0.88       274

    accuracy                           0.91     43937
   macro avg       0.94      0.85      0.89     43937
weighted avg       0.91      0.91      0.91     43937



In [11]:
import eli5
eli5.show_weights(crf, horizontal_layout=False)
